In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import seaborn as sns

# --- 定数定義 ---
H, R = 10.0, 5.0
N_LEAVES = 3000
N_LIGHTS = 100
N_SNOW = 300
np.random.seed(42)

# 1. 葉のデータ生成 (円錐サンプリング)
df_leaves = pd.DataFrame({
    'z': np.random.uniform(0, H, N_LEAVES),
    'theta': np.random.uniform(0, 2 * np.pi, N_LEAVES),
    'rand_r': np.sqrt(np.random.uniform(0, 1, N_LEAVES))
})
df_leaves['r'] = (R * (H - df_leaves['z']) / H) * df_leaves['rand_r']
df_leaves['x'] = df_leaves['r'] * np.cos(df_leaves['theta'])
df_leaves['y'] = df_leaves['r'] * np.sin(df_leaves['theta'])
df_leaves['intensity'] = df_leaves['z'] / H

# 共通設定 (ズームと背景を固定)
COMMON_CAMERA = dict(eye=dict(x=1.5, y=1.5, z=1.2), center=dict(x=0, y=0, z=-0.1))
LAYOUT_BASE = dict(
    width=640, height=640, template='plotly_dark', paper_bgcolor='black',
    showlegend=False, margin=dict(l=0, r=0, b=0, t=0),
    scene=dict(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False, aspectmode='data', camera=COMMON_CAMERA)
)

# 木の表示
fig1 = go.Figure(layout=LAYOUT_BASE)
fig1.add_trace(go.Scatter3d(
    x=df_leaves['x'], y=df_leaves['y'], z=df_leaves['z'],
    mode='markers',
    marker=dict(size=1.5, color=df_leaves['intensity'], colorscale='Greens', cmin=0, cmax=1, opacity=0.6, line=dict(width=0)),
    name='Leaves'
))
fig1.show()

In [ ]:
# 2. ライトのデータ生成
df_lights = pd.DataFrame({
    'z': np.random.uniform(0.1, H, N_LIGHTS),
    'theta': np.random.uniform(0, 2 * np.pi, N_LIGHTS),
    'freq': np.random.uniform(0.1, 0.3, N_LIGHTS),
    'phase': np.random.uniform(0, 2 * np.pi, N_LIGHTS)
})
palette = sns.hls_palette(6, l=0.5, s=1).as_hex()
df_lights['r'] = (R * (H - df_lights['z']) / H) + 0.2
df_lights['x'] = df_lights['r'] * np.cos(df_lights['theta'])
df_lights['y'] = df_lights['r'] * np.sin(df_lights['theta'])
df_lights['color'] = np.random.choice(palette, N_LIGHTS)

# 木のデータにライトを追加
fig2 = go.Figure(data=list(fig1.data), layout=LAYOUT_BASE)
fig2.add_trace(go.Scatter3d(
    x=df_lights['x'], y=df_lights['y'], z=df_lights['z'],
    mode='markers',
    marker=dict(size=4, color=df_lights['color'], opacity=1.0, line=dict(width=0)),
    name='Lights'
))
fig2.show()

In [ ]:
# 3. 雪のデータ生成
df_snow = pd.DataFrame({
    'z': np.random.uniform(0, H + 2, N_SNOW),
    'x': np.random.uniform(-R-2, R+2, N_SNOW),
    'y': np.random.uniform(-R-2, R+2, N_SNOW),
    'size': np.random.uniform(1, 4, N_SNOW)
})

# 木 + ライト のデータに雪を追加
fig3 = go.Figure(data=list(fig2.data), layout=LAYOUT_BASE)
fig3.add_trace(go.Scatter3d(
    x=df_snow['x'], y=df_snow['y'], z=df_snow['z'],
    mode='markers',
    marker=dict(size=df_snow['size'], color='white', opacity=0.8, line=dict(width=0)),
    name='Snow'
))
fig3.show()

In [ ]:
NUM_FRAMES = 50
frames = []
V_SNOW = (H + 2) / NUM_FRAMES
LIGHT_SIZE_MAX = 5

for f in range(NUM_FRAMES):
    z_snow_moved = (df_snow['z'] - (f * V_SNOW)) % (H + 2)
    val = np.sin(2 * np.pi * f * df_lights['freq'] + df_lights['phase'])
    light_mask = (val > 0).astype(float)

    frames.append(go.Frame(
        data=[
            # 葉:
            go.Scatter3d(x=df_leaves['x'], y=df_leaves['y'], z=df_leaves['z'], mode='markers',
                         marker=dict(size=1.5, color=df_leaves['intensity'],
                                     colorscale='Greens', opacity=0.6, line=dict(width=0))),

            # ライト:
            go.Scatter3d(x=df_lights['x'], y=df_lights['y'], z=df_lights['z'], mode='markers',
                         marker=dict(size=LIGHT_SIZE_MAX * light_mask, color=df_lights['color'],
                                     opacity=1.0, line=dict(width=0))),

            # 雪
            go.Scatter3d(x=df_snow['x'], y=df_snow['y'], z=z_snow_moved, mode='markers',
                         marker=dict(size=df_snow['size'], color='white', opacity=0.8, line=dict(width=0)))
        ],
        name=f"frame{f}"
    ))

fig_final = go.Figure(
    data=frames[0].data,
    layout=go.Layout(
        **LAYOUT_BASE, # 共通レイアウトを展開
        updatemenus=[{
            "type": "buttons",
            "buttons": [
                {"label": "Play", "method": "animate", "args": [None, {"frame": {"duration": 50, "redraw": True}, "fromcurrent": True, "transition": {"duration": 0}, "loop": True}]},
                {"label": "Pause", "method": "animate", "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]}
            ],
            "x": 0.05, "y": 0.05
        }]
    ),
    frames=frames
)

fig_final.show(config={"responsive": False})
fig_final.write_html("index.html")